In [1]:
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent

import time
import requests

In [2]:
UA = UserAgent()

HEADERS = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "user-agent": UA.random,
}

In [3]:
def extract_telegram(profile):
    contact_links = profile.find_all("a", class_="b")
    for link in [l.get("href") for l in contact_links]:    
        if link.startswith("tg://resolve?domain="):
            return link.replace("tg://resolve?domain=", "")
    return None

In [4]:
START_PAGE = f"https://www.intimcity.nl/search_ext.php?district%5B27%5D=on&district%5B30%5D=on&district%5B39%5D=on"

girl_list = requests.get(                
    START_PAGE,
    headers=HEADERS,
    timeout=30,
)

In [5]:
def extract_pages(page):
    links = []
    pages = page.find_all("a")
    for p in pages:
        link = p.get('href')
        if link.startswith('search_ext.php'):
            links.append("https://www.intimcity.nl/" + link)
    return links

In [6]:
def extract_accounts(page):
    links = []
    pages = page.find_all("a")
    for p in pages:
        link = p.get('href')
        if link.startswith('/persons.php'):
            links.append(link)
    return links

In [7]:
def extract_girls_telegrams_from_page(page):
    telegrams = []
    profiles = extract_accounts(page)
    for profile in profiles:
        try:
            girl_profile = requests.get(                
                f"https://www.intimcity.nl" + profile,
                headers=HEADERS,
                timeout=30,
            )
        except:
            continue
        
        telegram = extract_telegram(bs(girl_profile.text))
        if telegram is not None:
            print(telegram)
            telegrams.append(telegram)
        time.sleep(0.2)
    
    return telegrams

In [8]:
index_page = bs(girl_list.text)
pages = set([START_PAGE] + extract_pages(index_page))

print("Number of pages: ", len(pages))

all_telegrams = []

for i, page in enumerate(pages):

    print('parsing page', i, page)

    page_soup = requests.get(                
        page,
        headers=HEADERS,
        timeout=30,
    )

    all_telegrams += extract_girls_telegrams_from_page(bs(page_soup.text))
    time.sleep(0.2)


Number of pages:  7
parsing page 0 https://www.intimcity.nl/search_ext.php?district%5B27%5D=on&district%5B30%5D=on&district%5B39%5D=on
samira88095
assia98
mila576609
anitalove88
dashylichka2957
katya45267
diana78499
assia98
miria2999
theannL
kamila46621
mila576609
Lola7014
kamila46621
SexBombochka
diana78499
masha26799
eva78009
masha26799
parsing page 1 https://www.intimcity.nl/search_ext.php?text=&type[0]=&type[1]=&type[2]=&a1=&a2=&h1=&h2=&w1=&w2=&b1=&b2=&ws1=&ws2=&fs1=&fs2=&hs1=&hs2=&p11=&p12=&p21=&p22=&p31=&p32=&pn1=&pn2=&nm=&mm=&tr=0&check=&retouch=&indi=&vo=&lng=0&lat=0&r=&phone=&phones=&msgs=&st=&le=&chks=&pon=&hr=0&hu=0&wi=&wp=&wa=&wt=&ms=0&wt1=&wt2=&wt3=&cph=&mp=&ar=&int=&vs=&ex=&news=&sort=&dir=down&view=1&district[27]=on&district[30]=on&district[39]=on&index=3
karinneeeee
Valeria5322
yana08655
Lolitabum
Angelinsweet
veronika_blondi
sweeet_switch
miria2999
AdaMassage
lizabamby
parsing page 2 https://www.intimcity.nl/search_ext.php?text=&type[0]=&type[1]=&type[2]=&a1=&a2=&h1=&h

In [14]:
R="\n".join(sorted(set(all_telegrams)))
f = open('list/2022-18-11-sokol.csv','w')
f.write(R)
f.close()